# Simple Generative AI Model

This notebook demonstrates a complete generative AI model implementation in a single Python file. The model uses Hugging Face Transformers library to create a text generation system that runs on CPU.

## Overview
- Import required libraries
- Load and configure a pre-trained GPT-2 model
- Create a simple text generation function
- Test the model with sample inputs
- Measure performance metrics

## 1. Import Required Libraries

First, we import all the necessary libraries for our generative AI model.

In [1]:
# Import necessary libraries for the generative AI model
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import time
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set random seed for reproducible results
torch.manual_seed(42)

/Users/lalithyaalapati/Cloud_deploy/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/lalithyaalapati/Cloud_deploy/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Users/lalithyaalapati/Cloud_deploy/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Using device: cpu


## 2. Define the Complete Generative AI Model

This section contains the complete model implementation in a single Python class with all necessary methods.

In [2]:
class SimpleGenerativeAI:
    """
    A simple generative AI model using GPT-2 for text generation.
    This class handles model loading, text generation, and performance monitoring.
    """
    
    def __init__(self, model_name="gpt2"):
        """
        Initialize the generative AI model.
        
        Args:
            model_name (str): Name of the pre-trained model to use
        """
        print(f"Loading model: {model_name}")
        
        # Load the pre-trained tokenizer
        # Tokenizer converts text to tokens that the model can understand
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        
        # Add padding token if it doesn't exist
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Load the pre-trained model
        # This is the actual neural network that generates text
        self.model = GPT2LMHeadModel.from_pretrained(model_name)
        
        # Move model to the appropriate device (CPU or GPU)
        self.model.to(device)
        
        # Set model to evaluation mode for inference
        self.model.eval()
        
        print(f"Model loaded successfully on {device}")
    
    def generate_text(self, prompt, max_length=100, temperature=0.7, num_return_sequences=1):
        """
        Generate text based on a given prompt.
        
        Args:
            prompt (str): Input text to continue
            max_length (int): Maximum length of generated text
            temperature (float): Controls randomness (lower = more deterministic)
            num_return_sequences (int): Number of different outputs to generate
        
        Returns:
            list: Generated text sequences
        """
        # Record start time for performance measurement
        start_time = time.time()
        
        # Convert text prompt to tokens
        # Tokens are numerical representations that the model processes
        input_ids = self.tokenizer.encode(prompt, return_tensors='pt').to(device)
        
        # Generate text using the model
        with torch.no_grad():  # Disable gradient computation for faster inference
            outputs = self.model.generate(
                input_ids,
                max_length=max_length,
                temperature=temperature,
                num_return_sequences=num_return_sequences,
                do_sample=True,  # Enable sampling for more creative outputs
                pad_token_id=self.tokenizer.eos_token_id,
                no_repeat_ngram_size=2  # Prevent repetitive phrases
            )
        
        # Convert generated tokens back to text
        generated_texts = []
        for output in outputs:
            text = self.tokenizer.decode(output, skip_special_tokens=True)
            generated_texts.append(text)
        
        # Calculate inference time
        inference_time = time.time() - start_time
        
        # Print performance metrics
        print(f"Generation completed in {inference_time:.2f} seconds")
        print(f"Generated {len(generated_texts)} sequence(s)")
        
        return generated_texts
    
    def get_model_info(self):
        """
        Get information about the loaded model.
        
        Returns:
            dict: Model information including parameters and configuration
        """
        # Count the number of parameters in the model
        total_params = sum(p.numel() for p in self.model.parameters())
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        
        info = {
            "model_name": self.model.config.name_or_path,
            "total_parameters": total_params,
            "trainable_parameters": trainable_params,
            "vocab_size": self.tokenizer.vocab_size,
            "device": str(device)
        }
        
        return info

# Create an instance of our generative AI model
print("Initializing Simple Generative AI Model...")
gen_ai = SimpleGenerativeAI("gpt2")

Initializing Simple Generative AI Model...
Loading model: gpt2
Model loaded successfully on cpu


## 3. Display Model Information

Let's examine the details of our loaded model to understand its capabilities.

In [3]:
# Get and display model information
model_info = gen_ai.get_model_info()

print("Model Information:")
print("-" * 50)
for key, value in model_info.items():
    if "parameters" in key:
        # Format large numbers for better readability
        print(f"{key.replace('_', ' ').title()}: {value:,}")
    else:
        print(f"{key.replace('_', ' ').title()}: {value}")

print("-" * 50)

Model Information:
--------------------------------------------------
Model Name: gpt2
Total Parameters: 124,439,808
Trainable Parameters: 124,439,808
Vocab Size: 50257
Device: cpu
--------------------------------------------------


## 4. Test the Model with Sample Inputs

Now let's test our generative AI model with various prompts to see how it performs.

In [9]:
# Test Case 1: Simple story beginning
print("Test Case 1: Story Generation")
print("=" * 60)

prompt1 = "Once upon a time in a distant galaxy"
generated_text1 = gen_ai.generate_text(
    prompt=prompt1,
    max_length=80,
    temperature=0.5,
    num_return_sequences=1
)

print(f"Prompt: {prompt1}")
print(f"Generated: {generated_text1[0]}")
print("\n" + "=" * 60 + "\n")

# Test Case 2: Technical explanation
print("Test Case 2: Technical Content")
print("=" * 60)

prompt2 = "Artificial intelligence is"
generated_text2 = gen_ai.generate_text(
    prompt=prompt2,
    max_length=70,
    temperature=0.1,  # Lower temperature for more focused output
    num_return_sequences=1
)

print(f"Prompt: {prompt2}")
print(f"Generated: {generated_text2[0]}")
print("\n" + "=" * 60 + "\n")



Test Case 1: Story Generation
Generation completed in 3.44 seconds
Generated 1 sequence(s)
Prompt: Once upon a time in a distant galaxy
Generated: Once upon a time in a distant galaxy, an alien race had been born, but it was the only one that was able to create a living being. The race was called the Humanoid race, and they were the first human race to be born in the galaxy.

The Humanoids were a race of beings who had evolved into the humanoid form. They were known as the Humans,


Test Case 2: Technical Content
Generation completed in 1.27 seconds
Generated 1 sequence(s)
Prompt: Artificial intelligence is
Generated: Artificial intelligence is a new field of research that has been in the works for a while now, but it has now been demonstrated in a very early stage of development in an attempt to understand how it works.

The researchers say that the new technology could be used to help people with autism, and that it could also help them to better understand




## 5. Performance Analysis and Benchmarking

Let's analyze the performance of our model by testing different parameters and measuring execution time.

## 6. Interactive Testing

Try your own prompts with the model. Modify the prompt variable below to test different inputs.

In [22]:
# Interactive testing - modify this prompt to test your own inputs
custom_prompt = "Machine learning is is important skill in this job market"

print(f"Testing custom prompt: '{custom_prompt}'")
print("=" * 60)

# Generate text with custom settings
custom_result = gen_ai.generate_text(
    prompt=custom_prompt,
    max_length=100,          # Adjust length as needed
    temperature=0.5,         # Adjust creativity (0.1 = conservative, 1.0 = creative)
    num_return_sequences=1   # Number of different outputs
)

print(f"Input: {custom_prompt}")
print(f"Output: {custom_result[0]}")

print("\n" + "=" * 60)
print("Model testing completed successfully!")
print("The generative AI model is working and ready for deployment.")
print("=" * 60)

Testing custom prompt: 'Machine learning is is important skill in this job market'
Generation completed in 1.36 seconds
Generated 1 sequence(s)
Input: Machine learning is is important skill in this job market
Output: Machine learning is is important skill in this job market, and many of the jobs available to those who don't have a diploma are not a good fit for the job.

Here are some resources to help you learn how to apply for these jobs:
...


Model testing completed successfully!
The generative AI model is working and ready for deployment.
